In [1]:
import pandas as pd

# Load your dataset (adjust the path)
df = pd.read_csv(r"E:\eirpro\eirdata\questions.csv")

# Keep only the useful columns
df = df[['question1', 'question2']].dropna()

print("Dataset loaded. Shape:", df.shape)
df.head()


Dataset loaded. Shape: (404348, 2)


,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [5]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')   # <-- add this line


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\elgin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\elgin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [7]:
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def preprocess(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    tokens = word_tokenize(text)
    return tokens

# Apply preprocessing
df['q1_tokens'] = df['question1'].apply(preprocess)
df['q2_tokens'] = df['question2'].apply(preprocess)

df.head()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\elgin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,question1,question2,q1_tokens,q2_tokens
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv..."
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,"[what, is, the, story, of, kohinoor, kohinoor,...","[what, would, happen, if, the, indian, governm..."
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,"[how, can, i, increase, the, speed, of, my, in...","[how, can, internet, speed, be, increased, by,..."
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,"[why, am, i, mentally, very, lonely, how, can,...","[find, the, remainder, when, math2324math, is,..."
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,"[which, one, dissolve, in, water, quikly, suga...","[which, fish, would, survive, in, salt, water]"


In [9]:
from nltk import ngrams
from collections import defaultdict
import random

# Combine all tokenized questions into one list
all_tokens = df['q1_tokens'].tolist() + df['q2_tokens'].tolist()

# Flatten the list of lists
flat_tokens = [token for sublist in all_tokens for token in sublist]

# Create trigrams
trigrams = list(ngrams(flat_tokens, 3))

# Build the N-gram frequency dictionary
model = defaultdict(list)
for w1, w2, w3 in trigrams:
    model[(w1, w2)].append(w3)

print(f"Total trigrams: {len(trigrams)}")
print("Sample:", list(model.items())[:5])


Total trigrams: 8930347
Sample: [(('what', 'is'), ['the', 'the', 'web', 'best', 'the', 'the', 'the', 'the', 'java', 'the', 'your', 'a', 'purpose', 'the', 'the', 'the', 'your', 'the', 'abstract', 'the', 'the', 'the', 'the', 'it', 'the', 'the', 'our', 'the', 'the', 'the', 'it', 'the', 'the', 'the', 'the', 'the', 'it', 'a', 'my', 'the', 'the', 'the', 'the', 'the', 'in', 'cpagripcom', 'the', 'it', 'journalism', 'the', 'the', 'the', 'general', 'a', 'the', 'the', 'the', 'the', 'a', 'the', 'the', 'the', 'it', 'an', 'considered', 'similar', 'the', 'the', 'garbage', 'the', 'the', 'the', 'zika', 'the', 'an', 'the', 'an', 'the', 'bss', 'it', 'the', 'your', 'russian', 'the', 'the', 'your', 'the', 'the', 'solution', 'the', 'the', 'the', 'the', 'the', 'wrong', 'fermentation', 'the', 'the', 'the', 'the', 'the', 'the', 'latency', 'the', 'the', 'the', 'the', 'most', 'the', 'your', 'the', 'the', 'quoras', 'mathxmath', 'a', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'a', 'the', 'the', 'working', 'd

In [11]:
def suggest_next_word(text, n=5):
    tokens = nltk.word_tokenize(text.lower())
    if len(tokens) < 2:
        return ["Type at least two words..."]
    last_two = tuple(tokens[-2:])
    suggestions = model.get(last_two, [])
    if not suggestions:
        return ["No suggestions found"]
    return random.sample(suggestions, min(n, len(suggestions)))

# Example
print(suggest_next_word("how can"))


['i', 'i', 'i', 'i', 'i']


In [13]:
from collections import Counter
import random

# Build frequency-based trigram model
trigram_freq = defaultdict(Counter)
for w1, w2, w3 in trigrams:
    trigram_freq[(w1, w2)][w3] += 1

def predict_next_words(seed_text, num_words=5, top_k=5):
    tokens = nltk.word_tokenize(seed_text.lower())
    if len(tokens) < 2:
        return ["Type at least two words..."]
    
    last_two = tuple(tokens[-2:])
    next_word_candidates = trigram_freq.get(last_two, {})
    
    if not next_word_candidates:
        return ["No prediction available."]
    
    # Sort by frequency and add slight randomness for novelty
    sorted_candidates = sorted(
        next_word_candidates.items(),
        key=lambda x: x[1] + random.random() * 0.2,
        reverse=True
    )
    
    predictions = [w for w, _ in sorted_candidates[:top_k]]
    
    # Generate phrase completions
    completions = []
    for word in predictions:
        new_phrase = seed_text + " " + word
        completions.append(new_phrase)
    return completions

# Test it
print(predict_next_words("how can i", top_k=5))


['how can i get', 'how can i find', 'how can i make', 'how can i do', 'how can i learn']


In [15]:
import pickle

# Save your trigram model
with open("E:/eirpro/eirdata/model_data.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ Model saved as model_data.pkl")


✅ Model saved as model_data.pkl
